# Test: Data Owner (Client1) - SyftBox Mode

Automated test notebook for the Data Owner workflow with SyftBox encryption.
Run with: `jupyter execute sc_test_do.ipynb`

In [4]:
import beaver
beaver.dev() # install dev packages only in local dev mode
# beaver.show_debug()

syftbox-sdk not found at /Users/kj/Desktop/projects/biovault-desktop/biovault-desktop/syftbox-sdk/python
beaver not found at /Users/kj/Desktop/projects/biovault-desktop/biovault-desktop/biovault-beaver/python

Restart kernel to use new versions.


In [2]:
!uv pip install scikit-misc scanpy -q
import beaver
import scanpy as sc # really slow first load

In [3]:
bv = beaver.ctx()
session = bv.active_session()

ImportError: syftbox-sdk is required for SyftBox backend. Install with: pip install syftbox-sdk

In [ ]:
# session.open() # open in finder

In [ ]:
session

In [7]:
from beaver import Twin
# Create and publish Twin to session
number = Twin(
    private=1337,
    public=100,
    name="number",
)

In [8]:
# Publish to session (encrypted for peer only)
session.remote_vars["number"] = number

🌍 Using PUBLIC data from Twin 'number...'
📢 Published Twin 'number' (public side available at: /Users/madhavajay/dev/biovault-desktop/workspace3/biovault/sandbox/client2@sandbox.local/datasites/client2@sandbox.local/shared/biovault/sessions/c4a9ae0bd0f6/data/2b628e136e5d4c76b48a0f47b0125ed6.beaver)


In [9]:
bv.inbox()

InboxView(/Users/madhavajay/dev/biovault-desktop/workspace3/biovault/sandbox/client2@sandbox.local/datasites/client2@sandbox.local/shared/biovault): empty

In [10]:
request = bv.wait_for_request(number)
request

📬 Found existing request: request_add_one_for_res
   From: client1@sandbox.local


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
⚡ Action: add_one_result
   Request from: client1@sandbox.local
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📋 Function: add_one

⚙️  Static Parameters:
  • arg[0]: dict = {'_beaver_twin_ref': True, 'twin_id': 'a4a08761...

💡 Next Steps:
   .run_mock()   → Test on mock/public data (safe preview)
   .run()        → Execute on real/private data
   .run_both()   → Run on both mock & real for comparison
   .reject(msg)  → Decline this request

🆔 IDs: comp=3840dcf61c50... result=f44573a77fd2...
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [11]:
# session.open() # open in finder

In [12]:
request.run_both().approve()

🧪 Step 1/2: Testing on mock/public data...
🧪 Testing on mock/public data...
🔍 [post-execution] Figures to capture: 0
🔍 [post-execution] Already captured IDs: set()
🔒 Step 2/2: Executing on real/private data...
✓ Private result: int
✅ Approving result for: add_one_result (twin_id=f44573a77fd2..., comp_id=3840dcf61c50..., result_id=f44573a77fd2)
   Sending to: client1@sandbox.local
🔒 Using PRIVATE data from Twin 'add_one_result...'
✓ Result sent to session folder: /Users/madhavajay/dev/biovault-desktop/workspace3/biovault/sandbox/client2@sandbox.local/datasites/client2@sandbox.local/shared/biovault/sessions/c4a9ae0bd0f6
💡 They can load it with: bv.inbox()['add_one_result'].load()


PosixPath('/Users/madhavajay/dev/biovault-desktop/workspace3/biovault/sandbox/client2@sandbox.local/datasites/client2@sandbox.local/shared/biovault/sessions/c4a9ae0bd0f6/7124d3911eb24439b0929b2b9de90077.beaver')

In [13]:
assert False # some small bugs below on waiting on object timing

AssertionError: 

## Lets do some Bigger Data

In [ ]:
mock = beaver.sample_data.single_cell.mock.download()
mock

In [ ]:
real = beaver.sample_data.single_cell.real.download()
real

## Need to quickly fix the data

In [ ]:
fix_mock = str(mock).replace("mock", "fix")
# rename some cols
adata_sim = sc.read(mock)
adata_sim.obs.rename(columns={"pct_counts_in_top_50_genes": "pct_counts_mt"}, inplace=True)
adata_sim.obs.rename(columns={"group": "cell_type"}, inplace=True)
adata_sim.write_h5ad(fix_mock)

## Load data with sc like normal

In [ ]:
adata_private = sc.read(real)
adata_mock = sc.read(fix_mock)

In [ ]:
print(f"[DO] Loaded private: {adata_private.n_obs} cells")
print(f"[DO] Loaded mock: {adata_mock.n_obs} cells")

assert adata_private.n_obs > 0, "Private data is empty"
assert adata_mock.n_obs > 0, "Mock data is empty"

In [ ]:
# from beaver.runtime import TrustedLoader
# import anndata as ad

# # Register AnnData serialization
# @TrustedLoader.register(ad.AnnData)
# def anndata_serialize_file(obj, path):
#     obj.write_h5ad(path)

# @TrustedLoader.register(ad.AnnData)
# def anndata_deserialize_file(path):
#     return ad.read_h5ad(path)

# print("[DO] Registered AnnData loader")

In [ ]:
# Create and publish Twin to session
patient_sc = Twin(
    private=adata_private,
    public=adata_mock,
    name="patient_sc",
)

In [ ]:
patient_sc.can_send()

In [ ]:
# Publish to session (encrypted for peer only)
session.remote_vars["patient_sc"] = patient_sc
print("[DO] Published patient_sc Twin to session")

In [ ]:
# Track processed requests
processed_requests = set()

In [ ]:
bv.inbox()

In [ ]:
# Request 1: Violin plot
TIMEOUT = 60
print("[DO] Waiting for request 1/4 (violin)...")
bv.wait_for_message(timeout=TIMEOUT, poll_interval=1.0)

In [ ]:
# Find the first unprocessed computation request in inbox
inbox = bv.inbox()
request_envs = [e for e in inbox if e.name and e.name.startswith("request_") and e.name not in processed_requests]
assert len(request_envs) > 0, "No new request found in inbox"
request_env = request_envs[0]
print(f"[DO] Received: {request_env.name}")

# Load it - this injects the request into globals
request_env.load()
processed_requests.add(request_env.name)

# The loaded variable name is the envelope name
request_obj = globals()[request_env.name]
print(f"[DO] Running: {request_env.name}")

result = request_obj.run_both()
assert result is not None, "run_both() returned None"

result.approve()
print("[DO] ✓ Request 1 complete")

In [ ]:
result

In [ ]:
result.show_figures("both")

In [ ]:
# Request 2: Embedding
print("[DO] Waiting for request 2/4 (embedding)...")
bv.wait_for_message(timeout=TIMEOUT, poll_interval=1.0)

inbox = bv.inbox()
request_envs = [e for e in inbox if e.name and e.name.startswith("request_") and e.name not in processed_requests]
assert len(request_envs) > 0, "No new request found in inbox"
request_env = request_envs[0]
print(f"[DO] Received: {request_env.name}")

request_env.load()
processed_requests.add(request_env.name)
request_obj = globals()[request_env.name]
print(f"[DO] Running: {request_env.name}")

result = request_obj.run_both()
assert result is not None, "run_both() returned None"

result.approve()
print("[DO] ✓ Request 2 complete")

In [ ]:
# Inspect captured figures (mock/public and private)
result.show_figures('both')


In [ ]:
# Request 3: PCA variance
print("[DO] Waiting for request 3/4 (pca)...")
bv.wait_for_message(timeout=TIMEOUT, poll_interval=1.0)

inbox = bv.inbox()
request_envs = [e for e in inbox if e.name and e.name.startswith("request_") and e.name not in processed_requests]
assert len(request_envs) > 0, "No new request found in inbox"
request_env = request_envs[0]
print(f"[DO] Received: {request_env.name}")

request_env.load()
processed_requests.add(request_env.name)
request_obj = globals()[request_env.name]
print(f"[DO] Running: {request_env.name}")

result = request_obj.run_both()
assert result is not None, "run_both() returned None"

result.approve()
print("[DO] ✓ Request 3 complete")

In [ ]:
# Inspect captured figures (mock/public and private)
result.show_figures('both')

In [ ]:
# Request 4: UMAP
print("[DO] Waiting for request 4/4 (umap)...")
bv.wait_for_message(timeout=TIMEOUT, poll_interval=1.0)

inbox = bv.inbox()
request_envs = [e for e in inbox if e.name and e.name.startswith("request_") and e.name not in processed_requests]
assert len(request_envs) > 0, "No new request found in inbox"
request_env = request_envs[0]
print(f"[DO] Received: {request_env.name}")

request_env.load()
processed_requests.add(request_env.name)
request_obj = globals()[request_env.name]
print(f"[DO] Running: {request_env.name}")

result = request_obj.run_both()
assert result is not None, "run_both() returned None"

result.approve()
print("[DO] ✓ Request 4 complete")

In [ ]:
# Inspect captured figures (mock/public and private)
result.show_figures('both')

In [ ]:
# Final assertions
print("\n" + "="*50)
print("[DO] TEST PASSED")
print("[DO] Successfully handled 4 computation requests")
print("="*50)